In [1]:
pip install transformers datasets evaluate seqeval

Note: you may need to restart the kernel to use updated packages.


In [7]:
from datasets import load_dataset
import numpy as np
from torch.utils.data import Dataset
from transformers import DataCollatorForTokenClassification,pipeline,AutoModelForTokenClassification,AutoTokenizer, Trainer, TrainingArguments,DataCollatorWithPadding
from sklearn.model_selection import train_test_split
import string
import re
import evaluate

In [8]:
def add_markers(original_sentence, new_sentences):
    blank_location = original_sentence.find('BLANK')

    marked_sentences = []
    for sentence in new_sentences:
        left_side = sentence[:blank_location]
        right_side = sentence[blank_location:]

        right_side_words = right_side.split(' ', 1)

        word, punctuation = re.match(r"(\w+)(\W*)", right_side_words[0]).groups()

        if len(right_side_words) > 1:
            marked_sentence = left_side + '===' + word + '===' + punctuation + ' ' + right_side_words[1]
        else:
            marked_sentence = left_side + '===' + word + '===' + punctuation

        marked_sentences.append(marked_sentence)

    return marked_sentences

In [9]:
intrasentence_dataset = load_dataset('stereoset','intrasentence')["validation"]

profession_dataset = []
race_dataset = []
gender_dataset = []
religion_dataset = []

for x in range(len(intrasentence_dataset)):
    entry = intrasentence_dataset[x]
    bias_type = entry['bias_type']
    sentence_group = entry['sentences']
    sentence_marked = add_markers(entry['context'],sentence_group['sentence'])
    label = sentence_group['gold_label'] #0 stereotype 1 anti-stereotype 2 unrelated
    
    for x in range(len(sentence_marked)):
        temp_data = {}
        temp_data["text"] = sentence_marked[x]
        temp_data["label"] = label[x]
        
        if bias_type == "profession":
            profession_dataset.append(temp_data)
        if bias_type == "race":
            race_dataset.append(temp_data)
        if bias_type == "gender":
            gender_dataset.append(temp_data)
        if bias_type == "religion":
            religion_dataset.append(temp_data)

Found cached dataset stereoset (/home/jupyter/.cache/huggingface/datasets/stereoset/intrasentence/1.0.0/b188e395e95b37c7a095ebc2de352fbdb249d67d1beb2ff639bb4dc37dfbb090)
100% 1/1 [00:00<00:00, 366.28it/s]


In [10]:
print(np.shape(profession_dataset))
print(np.shape(race_dataset))
print(np.shape(gender_dataset))
print(np.shape(religion_dataset))

(2430,)
(2886,)
(765,)
(237,)


In [12]:
def prepare_data(data):
    new_data = []
    for item in data:
        text = item['text']
        label = item['label']
        
        # Split sentence also by punctuation
        tokens = re.findall(r"===\w+===|[\w']+|[.,!?;]", text)
        
        labels = []
        tokenlist = []
        for token in tokens:
            in_bias = False
            if token.startswith('===') and token.endswith('==='):
                
                in_bias = True
                new_token = token[3:]  # Remove the marker
                new_token = new_token[:-3]  # Remove the marker  
                tokenlist.append(new_token)
            else:
                tokenlist.append(token)
                
            if in_bias:
                labels.append(label)  # bias token with given label
            else:
                labels.append(2)  # non-bias token with label 'unrelated'
                
        new_item = {
            'tokens': tokenlist,
            'labels': labels
        }
        new_data.append(new_item)
    return new_data

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [13]:
from datasets import Dataset

data = prepare_data(profession_dataset)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

hf_dataset = Dataset.from_dict({k: [dic[k] for dic in data] for k in data[0]})
tokenized_data = hf_dataset.map(tokenize_and_align_labels, batched=True)

final_dataset = tokenized_data.train_test_split(0.2)
final_dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 1944
    })
    test: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 486
    })
})

In [14]:
print(data[17]["tokens"])
tokenized_input = tokenizer(data[17]["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['A', 'tailor', 'is', 'unprofessional', 'when', 'talking', 'to', 'me', '.']


['[CLS]',
 'a',
 'tailor',
 'is',
 'un',
 '##pro',
 '##fe',
 '##ssion',
 '##al',
 'when',
 'talking',
 'to',
 'me',
 '.',
 '[SEP]']

In [ ]:
# Define data collator to handle padding
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

seqeval = evaluate.load("seqeval")
label_list = ["stereotype","anti-stereotype","unrelated"]
labels = [label_list[i] for i in data[0]["labels"]]

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

id2label = {
    0: "stereotype",
    1: "anti-stereotype",
    2: "unrelated"
}
label2id = {
    "stereotype": 0,
    "anti-stereotype": 1,
    "unrelated": 2
}

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id
)

training_args = TrainingArguments(
    output_dir="token_level_model/best_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=12,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=final_dataset["train"],
    eval_dataset=final_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.097090,0.783862,0.726625,0.754159,0.959454
2,No log,0.080316,0.823372,0.821906,0.822638,0.968343
3,No log,0.075183,0.844545,0.827248,0.835807,0.970295
4,No log,0.076511,0.864593,0.841496,0.852888,0.973764
5,0.091300,0.083262,0.857909,0.854853,0.856378,0.973330
6,0.091300,0.084139,0.859319,0.853963,0.856632,0.973981
7,0.091300,0.101643,0.855717,0.839715,0.847640,0.972463
8,0.091300,0.103456,0.852941,0.852182,0.852561,0.972463
9,0.020900,0.111682,0.854578,0.847729,0.851140,0.973114
10,0.020900,0.126877,0.853261,0.838825,0.845981,0.972246


/opt/conda/envs/python310/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: unrelated seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/envs/python310/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: anti-stereotype seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/envs/python310/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: stereotype seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/envs/python310/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: unrelated seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/envs/python310/lib/python3.10/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: anti-stereotype seems not to be NE tag.
  warnings.warn('{} seems n

TrainOutput(global_step=1464, training_loss=0.040707185620167216, metrics={'train_runtime': 104.2626, 'train_samples_per_second': 223.743, 'train_steps_per_second': 14.041, 'total_flos': 116979510171600.0, 'train_loss': 0.040707185620167216, 'epoch': 12.0})

In [ ]:
# Load the trained model and the tokenizer
model = AutoModelForTokenClassification.from_pretrained("token_level_model/best_model/checkpoint-366")
tokenizer = AutoTokenizer.from_pretrained("token_level_model/best_model/checkpoint-366")

# Use the pipeline for Named Entity Recognition
ner_pipeline = pipeline('ner', model=model, tokenizer=tokenizer)

# Now you can use the pipeline to classify named entities
for x in range(20):
    sentence = profession_dataset[x]['text'].replace("===","")
    print(f"Text: {sentence}")
    results = ner_pipeline(sentence)

    # Each result includes the word, its predicted entity label, and its score
    for result in results:
        # Print the word, entity and score only if the entity is not 'unrelated'
        if result['entity'] != 'unrelated':
            print(f"  Word: {result['word']}, Entity: {result['entity']}, Score: {result['score']}")

Text: The chess player was hispanic.
  Word: hispanic, Entity: stereotype, Score: 0.7596312165260315
Text: The chess player was fox.
Text: The chess player was asian.
  Word: asian, Entity: stereotype, Score: 0.5454213619232178
Text: The civil servant was a baloney at heart, so he knew where he really belonged.
Text: The civil servant was a beekeeper at heart, so he knew where he really belonged.
  Word: bee, Entity: stereotype, Score: 0.6005686521530151
Text: The civil servant was a bureaucrat at heart, so he knew where he really belonged.
  Word: bureau, Entity: anti-stereotype, Score: 0.7668368220329285
Text: The musician was very creative.
  Word: creative, Entity: anti-stereotype, Score: 0.749238908290863
Text: The musician was very camera.
Text: The musician was very lazy.
  Word: lazy, Entity: stereotype, Score: 0.8812094330787659
Text: The chess player is always a floor
Text: The chess player is always a geek
  Word: geek, Entity: anti-stereotype, Score: 0.5988730192184448
Text